# Data Fetching

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pTPGMdx3qKbuXK_T9_rKDdd2rB5RJvuG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pTPGMdx3qKbuXK_T9_rKDdd2rB5RJvuG" -O fundus && rm -rf /tmp/cookies.txt

--2023-12-07 02:31:44--  https://docs.google.com/uc?export=download&confirm=t&id=1pTPGMdx3qKbuXK_T9_rKDdd2rB5RJvuG
Resolving docs.google.com (docs.google.com)... 74.125.204.139, 74.125.204.113, 74.125.204.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.204.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-5o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gp1bmeb5aip1d3kkabjs7hducvtqcepj/1701916275000/01516473392466462873/*/1pTPGMdx3qKbuXK_T9_rKDdd2rB5RJvuG?e=download&uuid=60ce9384-0765-4bb5-b57a-18dae8efe57e [following]
--2023-12-07 02:31:45--  https://doc-14-5o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gp1bmeb5aip1d3kkabjs7hducvtqcepj/1701916275000/01516473392466462873/*/1pTPGMdx3qKbuXK_T9_rKDdd2rB5RJvuG?e=download&uuid=60ce9384-0765-4bb5-b57a-18dae8efe57e
Resolving doc-14-5o-docs.googleusercontent.com (doc-14-5o-docs.googleusercontent.com)... 142.251.8.132, 2

In [ ]:
!unrar x 'fundus'


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from fundus

Creating    Training                                                  OK
Creating    Training/ARMD                                             OK
Extracting  Training/ARMD/1012.png                                         0%  OK 
Extracting  Training/ARMD/1013.png                                         0%  OK 
Extracting  Training/ARMD/1028.png                                         0%  OK 
Extracting  Training/ARMD/1029.png                                         0%  OK 
Extracting  Training/ARMD/1030.png                                         0%  OK 
Extracting  Training/ARMD/1050.png                                         0%  OK 
Extracting  Training/ARMD/1068.png                                         0%  OK 
Extracting  Training/ARMD/1083.png                                         0%  OK 
Extracting  Traini

# Feature Extraction


In [ ]:
import cv2
import numpy as np
from skimage.feature import graycomatrix
from skimage.filters import gabor
from skimage import color

def compute_texture_features(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute Haralick texture features
    haralick_features = np.mean(graycomatrix(gray_image, [5], [0], symmetric=True, normed=True), axis=(0, 1))

    # Compute Gabor texture features
    gabor_features = np.mean(np.abs(gabor(gray_image, frequency=0.6)[0]), axis=(0, 1))

    return np.concatenate((haralick_features, gabor_features), axis=None)

def compute_color_features(image):
    # Extract color histogram features
    color_histogram = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]).flatten()

    return color_histogram

def compute_statistical_features(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute mean, standard deviation, skewness, and kurtosis
    mean_value = np.mean(gray_image)
    std_deviation = np.std(gray_image)
    skewness = np.mean((gray_image - mean_value) ** 3) / (np.power(std_deviation, 3) + 1e-10)
    kurtosis = np.mean((gray_image - mean_value) ** 4) / (np.power(std_deviation, 4) + 1e-10)

    return np.array([mean_value, std_deviation, skewness, kurtosis])

def extract_combined_features(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Extract features
    texture_features = compute_texture_features(image)
    color_features = compute_color_features(image)
    statistical_features = compute_statistical_features(image)

    # Combine all features
    combined_features = np.concatenate((texture_features, color_features, statistical_features), axis=None)

    return combined_features

In [ ]:
import os

# Function to load images and labels from a directory with feature extraction
def load_images_and_features(folder):
    images = []
    labels = []

    for class_folder in os.listdir(folder):
        class_path = os.path.join(folder, class_folder)
        if os.path.isdir(class_path):
            img_paths = [os.path.join(class_path, filename) for filename in os.listdir(class_path) if filename.endswith('.png')]

            for img_path in img_paths:
              features = extract_combined_features(img_path)
              images.append(features)
              labels.append(class_folder)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

train_path = 'Training'
val_path = 'Validation'
test_path = 'Testing'

x_train, y_train = load_images_and_features(train_path)
# x_val, y_val = load_images_and_features(val_path)
x_test, y_test = load_images_and_features(val_path)

# Classifier

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Train a decision tree classifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)

# Evaluate the classifier
y_pred = dtc.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Decision Tree Classifier Accuracy: {:.2f}%".format(accuracy * 100))
print("F1: " + str(f1))

Decision Tree Classifier Accuracy: 65.46%
F1: 0.6475647997166358


## KNN

In [ ]:
# Import KNN from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Train a KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

# Evaluate the classifier
y_pred = knn.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("KNN Classifier Accuracy: {:.2f}%".format(accuracy * 100))
print("F1: " + str(f1))

KNN Classifier Accuracy: 66.12%
F1: 0.65657134781563


## Naive Bayes

In [ ]:
# Import Naive Bayes from sklearn
from sklearn.naive_bayes import GaussianNB

# Train a Naive Bayes classifier
nb = GaussianNB()
nb.fit(x_train, y_train)

# Evaluate the classifier
y_pred = nb.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Naive Bayes Classifier Accuracy: {:.2f}%".format(accuracy * 100))
print("F1: " + str(f1))

Naive Bayes Classifier Accuracy: 60.86%
F1: 0.565921570354393


## Random Forest

In [ ]:
# Import Random Forest from sklearn
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
rf_classifier.fit(x_train, y_train)

# Evaluate the classifier
y_pred = rf_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Random Forest Classifier Accuracy: {:.2f}%".format(accuracy * 100))
print("F1: " + str(f1))

Random Forest Classifier Accuracy: 69.74%
F1: 0.6503374792848478
